#### Data Cleanup 2



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import timedelta
import os
#from pandas_profiling import ProfileReport

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 1000

In [3]:
def generate_profile(df):
    '''generate pandas profile'''
 
    profile = ProfileReport(df)   
    profile.to_widgets()

In [4]:
file = (os.path.abspath(''))  
print(file)

#combined = pd.read_csv('/Users/Birr/projects/cap2/data/interim/combined.csv', low_memory=False)
combined = pd.read_csv('D:\cap\capstone2\data\interim\combined.csv', low_memory=False)

D:\cap\capstone2\notebooks


In [5]:
combined.head()

,Unnamed: 0,pilot_key,ev_id,ac_key,find_key,ev_date,far_part,second_pilot,crew_category,crew_age,crew_sex,med_certf,med_crtf_vldty,date_lst_med,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type,ev_country,mid_air,on_ground_collision,light_cond,ev_highest_injury,wx_cond_basic,flt_plan_filed,type_fly,phase_flt_spec,CERT_ATP,CERT_REC,CERT_CFI,CERT_COMM,CERT_FE,CERT_FORN,CERT_MIL,CERT_PRI,CERT_STU,CERT_SPRT,CERT_RMPT,CERT_NONE,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTR_AME,INSTR_ASE,INSTR_GLI,INSTR_GYRO,INSTR_HELI,INSTR_IAIR,INSTR_IHEL,INSTR_PLFT,INSTR_SPRT,INSTR_NONE,INSTRUM_APLN,INSTRUM_HELI,INSTRUM_PLFT,INSTRUM_NONE,OTHR_AIRS,OTHR_BALL,OTHR_GLI,OTHR_GYRO,OTHR_HELI,OTHR_PLFT,OTHR_SUAS,OTHR_NONE,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INST,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,GLDR-INST,GLDR-IRCV,GLDR-L24H,GLDR-L30D,GLDR-L90D,GLDR-PIC,GLDR-TOTL,INC-NO,LTA-INST,LTA-IRCV,LTA-L24H,LTA-L30D,LTA-L90D,LTA-PIC,LTA-TOTL,MAKE-INST,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INST,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INST,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,REL-NO,ROTO-INST,ROTO-IRCV,ROTO-L24H,ROTO-L30D,ROTO-L90D,ROTO-PIC,ROTO-TOTL,SENG-INST,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-INST,SIMU-IRCV,SIMU-L24H,SIMU-L30D,SIMU-L90D,SIMU-PIC,SIMU-TOTL
0,0,20001208X0880311,20001208X08803,20001208X088031,20001208X0880311,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,2,Personnel issues,4,Action/decision,15,Info processing/decision,20,Decision making/judgment,44,Pilot,C,ACC,USA,N,N,DAYL,FATL,IMC,NONE,PERS,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,20001208X0880311,20001208X08803,20001208X088031,20001208X0880312,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,84,Decision related to condition,C,ACC,USA,N,N,DAYL,FATL,IMC,NONE,PERS,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,20001208X0880311,20001208X08803,20001208X088031,20001208X0880313,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,50,Ceiling/visibility/precip,75,Below VFR minima,82,Effect on operation,C,ACC,USA,N,N,DAYL,FATL,IMC,NONE,PERS,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,20001208X0880311,20001208X08803,20001208X088031,20001208X0880314,9/14/1997 0:00:00,91,N,PILOT,46.0,M,CL3,UNK,10/3/1995 0:00:00,1,Aircraft,6,Aircraft oper/perf/capability,20,Performance/control parameters,12,Altitude,20,Not attained/maintained,C,ACC,USA,N,N,DAYL,F

In [6]:
combined['crew_age'].fillna(0, inplace=True)
combined['crew_age'] = combined['crew_age'].astype(int)

In [7]:
#subset pilots with age = 0  (no age reported)
zero_age = combined[combined['crew_age'] == 0]
zero_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3593 entries, 64 to 47608
Columns: 146 entries, Unnamed: 0 to SIMU-TOTL
dtypes: float64(111), int32(1), int64(6), object(28)
memory usage: 4.0+ MB


In [8]:
# fill in pilot age with average pilot age based on total flight hours

#define levels of pilot experience categories in bin of 250 hours
pilot0_max_hour = zero_age['ALL-TOTL'].max()
age_med_by_time = {} #holds median age for each time bracket
for hour_bin in range(1, int(pilot0_max_hour), 250):
    time_range = (hour_bin, hour_bin + 250)
    pop_age = combined[(combined['ALL-TOTL'] >= int(hour_bin)) & (combined['ALL-TOTL'] < (int(hour_bin) + 250))]
    age_med_by_time[time_range] = int(pop_age['crew_age'].median())
    
totl_idx = zero_age.columns.get_loc('ALL-TOTL')
age_idx = zero_age.columns.get_loc('crew_age')

for i in range(0, len(zero_age.index)):
    for key in age_med_by_time:
        if (zero_age.iloc[i,totl_idx] >= key[0]) & (zero_age.iloc[i,totl_idx] < key[1]):
            zero_age.iloc[i,age_idx] = age_med_by_time[key]
            

D:\Anaconda\envs\Capstone2\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [9]:
#aged_pilots contains pilots with imputed ages based on experience
aged_pilots = zero_age[zero_age['crew_age'] != 0]
aged_pilots.reset_index(inplace=True)
print(aged_pilots.info())
aged_pilots.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Columns: 147 entries, index to SIMU-TOTL
dtypes: float64(111), int32(1), int64(7), object(28)
memory usage: 112.3+ KB
None


,index,Unnamed: 0,pilot_key,ev_id,ac_key,find_key,ev_date,far_part,second_pilot,crew_category,crew_age,crew_sex,med_certf,med_crtf_vldty,date_lst_med,category_no,cat_descript,subcategory_no,sub_descript,section_no,sec_descript,subsection_no,subsec_descript,modifier_no,mod_descript,Cause_Factor,ev_type,ev_country,mid_air,on_ground_collision,light_cond,ev_highest_injury,wx_cond_basic,flt_plan_filed,type_fly,phase_flt_spec,CERT_ATP,CERT_REC,CERT_CFI,CERT_COMM,CERT_FE,CERT_FORN,CERT_MIL,CERT_PRI,CERT_STU,CERT_SPRT,CERT_RMPT,CERT_NONE,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTR_AME,INSTR_ASE,INSTR_GLI,INSTR_GYRO,INSTR_HELI,INSTR_IAIR,INSTR_IHEL,INSTR_PLFT,INSTR_SPRT,INSTR_NONE,INSTRUM_APLN,INSTRUM_HELI,INSTRUM_PLFT,INSTRUM_NONE,OTHR_AIRS,OTHR_BALL,OTHR_GLI,OTHR_GYRO,OTHR_HELI,OTHR_PLFT,OTHR_SUAS,OTHR_NONE,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INST,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,GLDR-INST,GLDR-IRCV,GLDR-L24H,GLDR-L30D,GLDR-L90D,GLDR-PIC,GLDR-TOTL,INC-NO,LTA-INST,LTA-IRCV,LTA-L24H,LTA-L30D,LTA-L90D,LTA-PIC,LTA-TOTL,MAKE-INST,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INST,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INST,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,REL-NO,ROTO-INST,ROTO-IRCV,ROTO-L24H,ROTO-L30D,ROTO-L90D,ROTO-PIC,ROTO-TOTL,SENG-INST,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-INST,SIMU-IRCV,SIMU-L24H,SIMU-L30D,SIMU-L90D,SIMU-PIC,SIMU-TOTL
0,1174,1174,20080729X0112411,20080729X01124,20080729X011241,20080729X0112411,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,3,Environmental issues,2,Physical environment,20,Object/animal/substance,10,Pole,91,Contributed to outcome,,INC,USA,N,N,DAYL,NONE,VMC,NONE,BUS,Approach - VFR pattern - turn to base,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1175,1175,20080729X0112411,20080729X01124,20080729X011241,20080729X0112412,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,2,Personnel issues,6,Task performance,10,Planning/preparation,10,Performance calculations,36,Flight crew,C,INC,USA,N,N,DAYL,NONE,VMC,NONE,BUS,Approach - VFR pattern - turn to base,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1176,1176,20080729X0112411,20080729X01124,20080729X011241,20080729X0112413,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,3,Environmental issues,3,Conditions/weather/phenomena,10,Temp/humidity/pressure,20,High density altitude,81,Effect on equipment,,INC,USA,N,N,DAYL,NONE,VMC,NONE,BUS,Approach - VFR pattern - turn to base,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1177,1177,20080729X0112411,20080729X01124,20080729X011241,20080729X0112414,7/28/2008 0:00:00,91,UNK,PILOT,60,M,CL2,VALID,5/15/2008 0:00:00,1,Aircraft,

In [10]:
#backfill ages to combined df.

age_idx = combined.columns.get_loc('crew_age')
pilot_idx = combined.columns.get_loc('pilot_key')

for i in range (0, len(aged_pilots.index)):
    for r in range (0, len(combined.index)):
        if aged_pilots.iloc[i,2] == combined.iloc[r,pilot_idx]:
            combined.iloc[r,age_idx] = aged_pilots.iloc[i,10]
           

In [11]:
#Check how many pilots left with no age derived
zero_age = combined[combined['crew_age'] == 0]
zero_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3495 entries, 64 to 47608
Columns: 146 entries, Unnamed: 0 to SIMU-TOTL
dtypes: float64(111), int32(1), int64(6), object(28)
memory usage: 3.9+ MB


In [12]:
#mean age of pilot in overall dataset
age_avg_df = combined[combined['crew_age']!=0]
avg_age = int(age_avg_df['crew_age'].mean())
print(avg_age)

53


In [13]:
#backfill remaining 0 ages with df average.
for i in range (0, len(combined.index)):
    if combined.iloc[i,age_idx] == 0:
        combined.iloc[i,age_idx] = avg_age

In [14]:
#Check how many pilots left with no age imputed
zero_age = combined[combined['crew_age'] == 0]
zero_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Columns: 146 entries, Unnamed: 0 to SIMU-TOTL
dtypes: float64(111), int32(1), int64(6), object(28)
memory usage: 0.0+ bytes


In [15]:
#table is now adjusted with derived ages
combined['crew_age'].describe()

count    47643.000000
mean        53.432571
std         14.768896
min         14.000000
25%         44.000000
50%         54.000000
75%         64.000000
max        100.000000
Name: crew_age, dtype: float64

In [16]:
#cleanup milestone - save to csv 

#combined.to_csv('/Users/Birr/projects/cap2/data/interim/df_aged.csv', index=False)
combined.to_csv('D:\cap\capstone2\data\interim\df_1.csv', index=False)